# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json

In [2]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

In [3]:
!cp driver.py flaskwebapp
!ls flaskwebapp

driver.py  etc	nginx


Below, we create the module for the Flask web application.

In [4]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [5]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [6]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [8]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Overwriting flaskwebapp/gunicorn_logging.conf


In [9]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [10]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [11]:
%%writefile flaskwebapp/requirements.txt

Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [12]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
    
RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 
    
RUN apt-get remove -y libcudnn7

RUN apt-get update && apt-get install -y --no-install-recommends \
        libcudnn7=7.0.5.15-1+cuda9.0 \
        libcudnn7-dev=7.0.5.15-1+cuda9.0 && \
    rm -rf /var/lib/apt/lists/* && \
    apt-get update

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython pandas jupyter ipykernel scikit-learn && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install tensorflow-gpu==1.6.0 && \
    pip install keras==2.1.6 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below referes to our dockerhub account. If you wish to push the image to your account make sure you change this.

In [13]:
image_name = "fboylu/kerasmnet-gpu"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [14]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon   16.9kB
Step 1/19 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
9.0-cudnn7-devel-ubuntu16.04: Pulling from nvidia/cuda

f539f7a1: Pulling fs layer 
2d420b43: Pulling fs layer 
bbeb6b91: Pulling fs layer 
2841ad7a: Pulling fs layer 
c6de4190: Pulling fs layer 
b970f65d: Pulling fs layer 
2d44e3fa: Pulling fs layer 
28d6c105: Pulling fs layer 
54b51c1e: Pulling fs layer 
460dc4c7: Pulling fs layer 
7eed712d: Pull complete 7.2MB/417.2MBBKExtracting  202.8MB/417.2MBDigest: sha256:85ab5de33649aaa5b9c2c02a1d6d20b454bdfc6489b3de9fb3e0a33656be2406
Status: Downloaded newer image for nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 3c7ce06587fb
Step 2/19 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in c68e8f5e5157
Removing intermediate container c68e8f5e5157
 ---> f5ef5cc1756e
Step 3/19 : USER root
 ---> Running in b495b1449a3a
Removing interm

 ---> Running in 571ecf9bac28
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Ign:3 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:4 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release [564 B]
Get:5 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release.gpg [801 B]
Get:6 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Packages [24.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release.gpg [801 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Packages [113 kB]


Get:40 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libxpm4 amd64 1:3.5.11-1ubuntu0.16.04.1 [33.8 kB]
Get:41 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgd3 amd64 2.1.1-4ubuntu0.16.04.8 [126 kB]
Get:42 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libxslt1.1 amd64 1.1.28-2.1ubuntu0.1 [145 kB]
Get:43 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx-common all 1.10.3-0ubuntu0.16.04.2 [26.6 kB]
Get:44 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx-core amd64 1.10.3-0ubuntu0.16.04.2 [428 kB]
Get:45 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx all 1.10.3-0ubuntu0.16.04.2 [3490 B]
Get:46 http://archive.ubuntu.com/ubuntu xenial/main amd64 python-pkg-resources all 20.7.0-1 [108 kB]
Get:47 http://archive.ubuntu.com/ubuntu xenial/universe amd64 python-meld3 all 1.0.2-2 [30.9 kB]
Get:48 http://archive.ubuntu.com/ubuntu xenial-updates/universe amd64 supervisor all 3.2.0-2ubuntu0.2 [253 kB]
debconf: d

Selecting previously unselected package libgd3:amd64.
Preparing to unpack .../libgd3_2.1.1-4ubuntu0.16.04.8_amd64.deb ...
Unpacking libgd3:amd64 (2.1.1-4ubuntu0.16.04.8) ...
Selecting previously unselected package libxslt1.1:amd64.
Preparing to unpack .../libxslt1.1_1.1.28-2.1ubuntu0.1_amd64.deb ...
Unpacking libxslt1.1:amd64 (1.1.28-2.1ubuntu0.1) ...
Selecting previously unselected package nginx-common.
Preparing to unpack .../nginx-common_1.10.3-0ubuntu0.16.04.2_all.deb ...
Unpacking nginx-common (1.10.3-0ubuntu0.16.04.2) ...
Selecting previously unselected package nginx-core.
Preparing to unpack .../nginx-core_1.10.3-0ubuntu0.16.04.2_amd64.deb ...
Unpacking nginx-core (1.10.3-0ubuntu0.16.04.2) ...
Selecting previously unselected package nginx.
Preparing to unpack .../nginx_1.10.3-0ubuntu0.16.04.2_all.deb ...
Unpacking nginx (1.10.3-0ubuntu0.16.04.2) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../python-pkg-resources_20.7.0-1_all.deb ...
Unp

Get:2 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  libcudnn7-dev 7.0.5.15-1+cuda9.0 [92.9 MB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 195 MB in 3s (55.5 MB/s)
Selecting previously unselected package libcudnn7.
(Reading database ... 16102 files and directories currently installed.)
Preparing to unpack .../libcudnn7_7.0.5.15-1+cuda9.0_amd64.deb ...
Unpacking libcudnn7 (7.0.5.15-1+cuda9.0) ...
Selecting previously unselected package libcudnn7-dev.
Preparing to unpack .../libcudnn7-dev_7.0.5.15-1+cuda9.0_amd64.deb ...
Unpacking libcudnn7-dev (7.0.5.15-1+cuda9.0) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Setting up libcudnn7 (7.0.5.15-1+cuda9.0) ...
Setting up libcudnn7-dev (7.0.5.15-1+cuda9.0) ...
update-alternatives: using /usr/include/x86_64-linux-gnu/cudnn_v7.h to provide /usr/include/cudnn.h (libcudnn) in auto mode
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Ign:1 http://develo

decorator 4.3.0########## | 100% 
python-dateutil 2.7.3########## | 100% 
pickleshare 0.7.4########## | 100% 
webencodings 0.5.1########## | 100% 
html5lib 1.0.1########## | 100% 
jupyter 1.0.0########## | 100% 
gmp 6.1.2########## | 100% 
gst-plugins-base 1.14.0########## | 100% 
numpy 1.14.3########## | 100% 
ipython 6.4.0########## | 100% 
expat 2.2.5########## | 100% 
jedi 0.12.0########## | 100% 
scipy 1.1.0########## | 100% 
ipykernel 4.8.2########## | 100% 
notebook 5.5.0########## | 100% 
pandas 0.23.0########## | 100% 
libsodium 1.0.16########## | 100% 
wcwidth 0.1.7########## | 100% 
pyqt 5.9.2########## | 100% 
traitlets 4.3.2########## | 100% 
jupyter_console 5.2.0########## | 100% 
ipython_genutils 0.2.0########## | 100% 
simplegeneric 0.8.1########## | 100% 
pandocfilters 1.4.2########## | 100% 
jupyter_core 4.4.0########## | 100% 
libxcb 1.13########## | 100% 
pcre 8.42########## | 100% 
jupyter_client 5.2.3########## | 100% 
icu 58.2########## | 100% 
gstreamer 1.14.0##

Removed requests-2.18.4-py36he2e5f8d_1.tar.bz2
Removed libgcc-ng-7.2.0-hdf63c60_3.tar.bz2
Removed qtconsole-4.3.1-py35h4626a06_0.tar.bz2
Removed python-3.5.5-hc3d631a_4.tar.bz2
Removed libgfortran-ng-7.2.0-hdf63c60_3.tar.bz2
Removed fontconfig-2.12.6-h49f89f6_0.tar.bz2
Removed ipykernel-4.8.2-py35_0.tar.bz2
Removed testpath-0.3.1-py35had42eaf_0.tar.bz2
Removed jupyter_client-5.2.3-py35_0.tar.bz2
Removed yaml-0.1.7-had09818_2.tar.bz2
Removed mistune-0.8.3-py35h14c3975_1.tar.bz2
Removed jpeg-9b-h024ee3a_2.tar.bz2
Removed gst-plugins-base-1.14.0-hbbd80ab_1.tar.bz2
Removed libxml2-2.9.8-h26e45fe_1.tar.bz2
Removed mkl_random-1.0.1-py35h629b387_0.tar.bz2
Removed decorator-4.3.0-py35_0.tar.bz2
Removed glib-2.56.1-h000015b_0.tar.bz2
Removed jupyter_core-4.4.0-py35ha89e94b_0.tar.bz2
Removed entrypoints-0.2.3-py35h48174a2_2.tar.bz2
Removed pyzmq-17.0.0-py35h14c3975_0.tar.bz2
Removed qt-5.9.5-h7e424d6_0.tar.bz2
Removed expat-2.2.5-he0dffb1_0.tar.bz2
Removed intel-openmp-2018.0.3-0.tar.bz2
Removed

  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a0/f8/e9/1933dbb3447ea6ef57062fd5461cb118deb8c2ed074e8344bf
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built absl-py termcolor gast html5lib
mkl-fft 1.0.0 requires cython, which is not installed.
mkl-random 1.0.1 requi

      Successfully uninstalled h5py-2.8.0
Cache location: 
There are no tarballs to remove
Removing intermediate container 6121315578a9
 ---> 2770393c0a78
Step 16/19 : EXPOSE 8888
 ---> Running in 1351d121ee9d
Removing intermediate container 1351d121ee9d
 ---> 39becdd10012
Step 17/19 : EXPOSE 5000
 ---> Running in ac1a6d5d6de1
Removing intermediate container ac1a6d5d6de1
 ---> e911b3a6e590
Step 18/19 : EXPOSE 80
 ---> Running in ea6339af8a63
Removing intermediate container ea6339af8a63
 ---> 2c5cc2f5c830
Step 19/19 : CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]
 ---> Running in da749be3c10b
Removing intermediate container da749be3c10b
 ---> 0efba6bc87b0
Successfully built 0efba6bc87b0
Successfully tagged fboylu/kerasmnet-gpu:latest


In [15]:
!docker push $image_name

The push refers to repository [docker.io/fboylu/kerasmnet-gpu]

127c5b7e: Preparing 
1f4d467c: Preparing 
63c4668f: Preparing 
2ba23067: Preparing 
9f931673: Preparing 
319c0122: Preparing 
a93a8ec6: Preparing 
9c60a426: Preparing 
a897e78a: Preparing 
ab55aa8c: Preparing 
e046f000: Preparing 
148cae00: Preparing 
cfebc71a: Preparing 
07dad4ba: Preparing 
9ebf93f3: Preparing 
91e51d73: Preparing 
d9e65295: Preparing 
45e78935: Preparing 
1dc646ba: Preparing 


3c4668f: Pushing  585.8MB/610MBGB [17A

f4d467c: Pushed   2.081GB/2.009GBPushing  1.876GB/2.009GBlatest: digest: sha256:585226ce30876ff7b7762f9ea23240f42a77ec1bb071a42f8c0f6a31483cdcf8 size: 4513


In [16]:
print('Docker image name {}'.format(image_name))

Docker image name fboylu/kerasmnet-gpu


We can now [test our image locally](03_TestLocally.ipynb).